In [81]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate

In [82]:
dimension_2D = 20

num_classes = 20

In [83]:
import numpy as np

# Dataset generator - generating matrices
dimension_2D = 20
n_samples= 21
start=0
end=20

# Extracting the distance matrix
f1=open('Dataset1_final.txt','r')
l1=[]
l1=[line.split() for line in f1]
distance = np.array(l1,dtype = object)
distance = distance.reshape((10000,20,20))

# Extracting the adjency matrix without distance
f2=open('Dataset2_final.txt','r')
l2=[]
l2=[line.split() for line in f2]
network = np.array(l1,dtype = object)
network = network.reshape((10000,20,20))

# Extracting the number of nodes and adding as labels
# Data distance is a 1000*2 matrix which contains 1000 20*20*1 matrices
f3=open('Dataset_Labels_final.txt','r')
labels=[l.split() for l in f3]

# Adding labels to the distance matrix
data_distance = []
data_distance_labels = []
for i in range(1000):
    data_distance.append((distance[i]))
    data_distance_labels.append((int(labels[i][0])))
data_distance = np.array(data_distance,dtype=object)
data_distance_labels = np.array(data_distance_labels,dtype=object)
data_distance_labels = data_distance_labels.astype('float32')

# Adding labels to network matrix
# Data network is a 1000*2 matrix which contains 1000 20*20*1 matrices

data_network = []
data_network_labels=[]
for i in range(10000):
    data_network.append(network[i])
    data_network_labels.append((int(labels[i][0])))
data_network = np.array(data_network,dtype=object)
data_network_labels = np.array(data_network_labels)
data_network = data_network.astype("float32")

In [84]:
# define the standalone discriminator model
def define_discriminator(in_shape=(dimension_2D,dimension_2D,1), n_classes=num_classes):
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 100)(in_label)
	# scale up to image dimensions with linear activation
	n_nodes = in_shape[0] * in_shape[1]
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((in_shape[0], in_shape[1], 1))(li)
	# image input
	in_image = Input(shape=in_shape)
	# concat label as a channel
	merge = Concatenate()([in_image, li])
	# downsample
	fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
	fe = LeakyReLU(alpha=0.2)(fe)
	# downsample
	fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	# flatten feature maps
	fe = Flatten()(fe)
	# dropout
	fe = Dropout(0.4)(fe)
	# output
	out_layer = Dense(1, activation='sigmoid')(fe)
	# define model
	model = Model([in_image, in_label], out_layer)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [85]:
# define the standalone generator model
def define_generator(latent_dim, n_classes=20):
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 50)(in_label)
	# linear multiplication
	n_nodes = 5 * 5
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((5, 5, 1))(li)
	# image generator input
	in_lat = Input(shape=(latent_dim,))
	# foundation for 5x5 image
	n_nodes = 128 * 5 * 5
	gen = Dense(n_nodes)(in_lat)
	gen = LeakyReLU(alpha=0.2)(gen)
	gen = Reshape((5, 5, 128))(gen)
	# merge image gen and label input
	merge = Concatenate()([gen, li])
	# upsample to 10x10
	gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
	gen = LeakyReLU(alpha=0.2)(gen)
	# upsample to 20x20
	gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
	gen = LeakyReLU(alpha=0.2)(gen)
	# output
	out_layer = Conv2D(1, (5,5), activation='tanh', padding='same')(gen)
	# define model
	model = Model([in_lat, in_label], out_layer)
	return model

In [86]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# get noise and label inputs from generator model
	gen_noise, gen_label = g_model.input
	# get image output from the generator model
	gen_output = g_model.output
	# connect image output and label input from generator as inputs to discriminator
	gan_output = d_model([gen_output, gen_label])
	# define gan model as taking noise and label and outputting a classification
	model = Model([gen_noise, gen_label], gan_output)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [87]:
# load fashion mnist images
def load_real_samples():
  trainX = data_network
  trainy = data_network_labels
  X = expand_dims(trainX, axis=-1)
  # convert from ints to floats
  X = X.astype('float32')
  # scale from [0,255] to [-1,1]
  X = (X - 127.5) / 127.5
  return [X, trainy]
 
 
# # select real samples
def generate_real_samples(dataset, n_samples):
  # split into images and labels
  images, labels = dataset
  # choose random instances
  ix = randint(0, images.shape[0], n_samples)
  # select images and labels
  X, labels = images[ix], labels[ix]
  # generate class labels
  y = ones((n_samples, 1))
  return [X, labels], y

In [88]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=20):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input, labels_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator.predict([z_input, labels_input])
	# create class labels
	y = zeros((n_samples, 1))
	return [images, labels_input], y

In [89]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=500, n_batch=128):
	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	g_loss_array = []
	d_loss1_array = []
	d_loss2_array = []
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
        # g_loss, d_loss1, d_loss2
		g_loss_avg = 0
		d_loss1_avg = 0
		d_loss2_avg = 0
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			[X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
			# generate 'fake' examples
			[X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
			# prepare points in latent space as input for the generator
			[z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
			g_loss_avg += g_loss
			d_loss1_avg += d_loss1
			d_loss1_avg += d_loss2
		g_loss_array.append(i,g_loss_avg)
		d_loss1_array.append(i,d_loss1_avg)
		d_loss2_array.append(i,d_loss2_avg)
	# save the generator model
	g_model.save('cgan_generator.h5')

In [90]:
# example of training an conditional gan on the fashion mnist dataset

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


2/2 [==============================] - 0s 6ms/step
>1, 1/6, d1=0.714, d2=0.697 g=0.690
2/2 [==============================] - 0s 5ms/step
>1, 2/6, d1=0.609, d2=0.711 g=0.675
2/2 [==============================] - 0s 13ms/step
>1, 3/6, d1=0.528, d2=0.734 g=0.654
2/2 [==============================] - 0s 5ms/step
>1, 4/6, d1=0.464, d2=0.769 g=0.623
2/2 [==============================] - 0s 5ms/step
>1, 5/6, d1=0.409, d2=0.817 g=0.583
2/2 [==============================] - 0s 7ms/step
>1, 6/6, d1=0.381, d2=0.885 g=0.543
2/2 [==============================] - 0s 5ms/step
>2, 1/6, d1=0.349, d2=0.951 g=0.508
2/2 [==============================] - 0s 5ms/step
>2, 2/6, d1=0.350, d2=1.002 g=0.487
2/2 [==============================] - 0s 5ms/step
>2, 3/6, d1=0.362, d2=1.019 g=0.481
2/2 [==============================] - 0s 5ms/step
>2, 4/6, d1=0.389, d2=1.011 g=0.497
2/2 [==============================] - 0s 5ms/step
>2, 5/6, d1=0.425, d2=0.985 g=0.520
2/2 [==============================] - 0s 

>10, 6/6, d1=0.638, d2=0.634 g=0.796


In [1]:
# example of loading the generator model and generating images
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.models import load_model
from matplotlib import pyplot

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=20):
 # generate points in the latent space
 x_input = randn(latent_dim * n_samples)
 # reshape into a batch of inputs for the network
 z_input = x_input.reshape(n_samples, latent_dim)
 # generate labels
 labels = randint(0, n_classes, n_samples)
 return [z_input, labels]

# create and save a plot of generated images
def save_plot(examples, n):
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
	pyplot.show()
# load model
model = load_model('cgan_generator.h5')
# generate images
latent_points, labels = generate_latent_points(latent_dim,100)
# specify labels
labels = asarray([x for _ in range(10) for x in range(10)])
# generate images
X  = model.predict([latent_points, labels])
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0

2022-12-14 19:12:59.139939: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-14 19:12:59.140009: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


OSError: No file or directory found at cgan_generator.h5

In [ ]:
save_plot(X,2)